# ML Enablement Workshop: サービスの解約率改善シナリオ クラウド活用編

## コンテンツ

1. 背景
1. 環境構築
1. 学習をスケールする
1. モデルをホスティングする
   1. 性能評価
   1. エンドポイントの削除
1. Notebookを移行する

---

## 1.背景

サービスの解約率を改善するために、 Studio Lab では機能的・コンピューティングリソース的に不十分な状況に直面することがあるかもしれません。例えば、重要なデータは Studio Lab に持ち出せないかもしれませんし、モデルを学習する、本番同等のトランザクションで検証するのに Studio Lab では力不足かもしれません。本Notebookでは、 Studio Lab では不十分な状況に直面した時に Amazon SageMaker を使用し機械学習の価値検証を継続する方法を解説します。 Studio Lab には AWS の機能を呼び出す AWS SDK がインストール済みで、 SageMaker への Notebook の移行を行う方法も整備されています。

![Studio Lab to SageMaker](images/sagemaker_001.png)

---

## 2.環境構築

### 2.1 Studio Lab の環境構築

本 Notebook を動かすための環境構築手順は本体のシナリオと同じため、先に [customer_churn.ipynb](./customer_churn.ipynb) を参照し環境構築を行ってください。
Jupyter Notebookの右上にある虫の隣のボタンをクリックしカーネルを切り替えます。

![environment_setup_002.png](./images/002.png)

### 2-2. AWS へ接続するための環境構築

AWSへアクセスするためのユーザー (IAM ユーザー) を作成します。IAM ユーザーの作成方法は以下のページを参考にします。名前は任意ですが、以降では`sagemaker-studio-lab-access`として扱います。

- https://docs.aws.amazon.com/ja_jp/IAM/latest/UserGuide/id_users_create.html#id_users_create_console

![sagemaker_002.png](./images/sagemaker_002.png)

プログラムによるアクセスを選択することと、[Set permissions] (アクセス許可の設定) のところでは、「既存のポリシーを直接アタッチ」を選択して下記2つのポリシーを選択するようにしてください。本ハンズオンでは簡易化のため IAM ユーザーの作成の際に、以下のポリシーを付与します。本番環境で実行する際は、必要最小権限にする必要があります。

- PowerUserAccess
- AmazonSageMakerFullAccess

IAM ユーザー作成時の最後のページで取得できるアクセスキーとシークレットアークセスキーをメモにコピーします。 **クレデンシャル情報の取り扱いには注意してください。** Studio Lab から `sagemaker-studio-lab-access` でアクセスできるように設定を行います。Terminalを起動します。

![sagemaker_003.png](./images/sagemaker_003.png)

ターミナル内で以下のコマンド実行します。
```
aws configure
```

アクセスキー ID とシークレットアクセスキーは先にメモをした内容を貼り付けます。リージョンには`ap-northeast-1`を入力し、アウトプットフォーマットは空のままエンターを押します。

![sagemaker_004.png](./images/sagemaker_004.png)


### 2-3. SageMaker Training Instance が利用する IAM ロールを作成する

学習を実行するインスタンスの権限となる、 IAM ロールを作成します。 IAM ロールの作成方法は以下のページを参考にします。名前は任意ですが、以降では`SageMakerStudioLabExecuteRole`として扱います。 

- https://docs.aws.amazon.com/ja_jp/glue/latest/dg/create-an-iam-role-sagemaker-notebook.html

![sagemaker_005.png](./images/sagemaker_005.png)

IAM ロールの作成の際に、以下のポリシーを付与されていることを確認します。
- AmazonSageMakerFullAccess

作成した IAM ロールの ARN をコピーし、以下のセルに入力します。マネジメントコンソールで IAM ロールから確認できます。

![sagemaker_006.png](./images/sagemaker_006.png)

In [1]:
role = "arn:aws:iam::000000000000:role/SageMakerStudioLabExecuteRole"

では、はじめていきましょう、はじめに利用するライブラリを読み込んでおきます。

In [2]:
# Define IAM role
from pathlib import Path
import boto3
import sagemaker
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


data_root = Path("../../data/")

---
## 3.学習をスケールする

Studio Lab の GPU では性能、稼働時間が足りない場合 AWS で学習を行うことができます。学習を始める前に、学習データをAmazon S3にアップロードしSageMakerから利用できるようにします。

※事前に `customer_churn.ipynb` のシナリオを実行しデータを作成しておく必要があります。

In [3]:
sagemaker_session = sagemaker.Session()
input_train = sagemaker_session.upload_data(path=str(data_root.joinpath('interim/churn_train.csv')), key_prefix='sagemaker/DEMO-xgboost-churn')
input_validation = sagemaker_session.upload_data(path=str(data_root.joinpath('interim/churn_validation.csv')), key_prefix='sagemaker/DEMO-xgboost-churn')

`input_train` と `input_validation` にはアップロードしたファイルのS3パスが保存されています。これらは csv ファイルで、学習させるには以下のようなデータである必要がありますが、先の前処理の段階でこのようなデータ形式に変換しているため、追加の処理は必要ありません。

- 1列目が予測対象のデータ
- ヘッダ行はなし

学習に使ったモデルは XGBoost でしたので、 Amazon SageMaker が用意している XGBoost のコンテナを利用して学習します。このコンテナは、ファイルをデフォルトで libsvm 形式と認識するため、`TrainingInput`という関数を利用して、`content_type='text/csv'`を明示的に指定します。

In [4]:
# from sagemaker.session import s3_input
from sagemaker.inputs import TrainingInput

content_type='text/csv'
s3_input_train = TrainingInput(input_train, content_type=content_type)
s3_input_validation = TrainingInput(input_validation, content_type=content_type)

Amazon SageMaker は、マネージドで、分散学習が設定済みで、リアルタイム推論のためのホスティングも可能な XGBoost コンテナを用意しています。 リージョンごと、アルゴリズムごとに用意されているコンテナの URI は [Docker レジストリパスとサンプルコード](https://docs.aws.amazon.com/ja_jp/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html)で確認できます。XGBoost のコンテナの場所を取得しましょう。

In [5]:
container = sagemaker.image_uris.retrieve("xgboost", boto3.Session().region_name, "1.2-1")
container

'354813040037.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-xgboost:1.2-1'

それでは学習を始めましょう。学習のためにハイパーパラメータを指定したり、学習のインスタンスの数やタイプを指定することができます。XGBoost における主要なハイパーパラメータは以下のとおりです。

- `max_depth` アルゴリズムが構築する木の深さをコントロールします。深い木はより学習データに適合しますが、計算も多く必要で、overfiting になる可能性があります。たくさんの浅い木を利用するか、少数の深い木を利用するか、モデルの性能という面ではトレードオフがあります。
- `subsample` 学習データのサンプリングをコントロールします。これは overfitting のリスクを減らしますが、小さすぎるとモデルのデータが不足してしまいます。
-  `num_round` ブースティングを行う回数をコントロールします。以前のイテレーションで学習したときの残差を、以降のモデルにどこまで利用するかどうかを決定します。多くの回数を指定すると学習データに適合しますが、計算も多く必要で、overfiting になる可能性があります。
- `eta` 各ブースティングの影響の大きさを表します。大きい値は保守的なブースティングを行います。
- `gamma` ツリーの成長の度合いをコントロールします。大きい値はより保守的なモデルを生成します。

XGBoostのhyperparameterに関する詳細は [GitHub](https://github.com/dmlc/xgboost/blob/master/doc/parameter.rst) もチェックしてください。

In [6]:
sess = sagemaker.Session()

hyperparameters = {"max_depth":"5",
                        "eta":"0.2",
                        "gamma":"4",
                        "min_child_weight":"6",
                        "subsample":"0.8",
                        "objective":"binary:logistic",
                        "num_round":"100"}

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    hyperparameters=hyperparameters,
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge',
                                    sagemaker_session=sess)

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation}) 

2022-09-15 12:22:20 Starting - Starting the training job...ProfilerReport-1663244539: InProgress
...
2022-09-15 12:23:13 Starting - Preparing the instances for training.........
2022-09-15 12:24:45 Downloading - Downloading input data...
2022-09-15 12:25:12 Training - Downloading the training image.........
2022-09-15 12:26:48 Uploading - Uploading generated training model[2022-09-15 12:26:36.718 ip-10-0-172-176.ap-northeast-1.compute.internal:1 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimit

学習ジョブは AWS Console からも確認できます。

![sagemaker_007.png](images/sagemaker_007.png)

学習したモデルは S3 に格納されています。

In [7]:
xgb.model_data

's3://sagemaker-ap-northeast-1-585936743357/sagemaker-xgboost-2022-09-15-12-22-19-369/output/model.tar.gz'

---
## 4.モデルをホスティングする

A/B テストを行う場合などは、別々に学習したモデルを API サーバーとして立てる必要があるかもしれません。 SageMaker では、学習が終われば`deploy()`を実行することで、エンドポイントを作成してモデルをデプロイできます。

In [8]:
xgb_predictor = xgb.deploy(initial_instance_count=1, instance_type = 'ml.m4.xlarge')

-------!

デプロイしたモデルは AWS Console から確認できます。

![sagemaker_008.png](images/sagemaker_008.png)

### 4-1.性能評価

ホスティングしたモデルを使用し、簡単に予測を行うことができます。予測は http の POST の request を送るだけです。
endpoint は `numpy` の `array` を受け取ることができないため、[`CSVSerializer`](https://sagemaker.readthedocs.io/en/stable/api/inference/serializers.html#sagemaker.serializers.CSVSerializer) を設定して `numpy` の `array` を csv 形式に変換して送ります。 逆に、endpoint から取得する時は csv からリストに変換します。

In [9]:
xgb_predictor.serializer = sagemaker.serializers.CSVSerializer()
xgb_predictor.deserializer = sagemaker.deserializers.CSVDeserializer()

先のノートブックで作成済みのテストデータを受け取ると、これをデフォルト500行ずつのデータにわけて、エンドポイントに送信する `predict` という関数を用意します。あとは `predict` を実行して予測結果を受け取ります。 

In [10]:
def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = []
    for array in split_array:
        predictions.append(np.array(xgb_predictor.predict(array), dtype=np.float32))

    return np.concatenate(predictions, axis=1)

test_data = pd.read_csv(data_root.joinpath('interim/churn_test.csv'), header=None)
predictions = predict(test_data.values[:, 1:])  # 0列目はラベルのため除外

機械学習の性能を比較評価する方法はいくつかありますが、単純に、予測値と実際の値を比較しましょう。今回は、顧客が離反する `1` と離反しない `0` を予測しますので、この混同行列を作成します。

In [11]:
pd.crosstab(index=test_data.iloc[:, 0], columns=np.round(predictions), rownames=['actual'], colnames=['predictions'])

predictions,0.0,1.0
actual,,
0,235,18
1,11,236


_注意点, アルゴリズムにはランダムな要素があるので結果は必ずしも一致しません._

評価の方法は、本体の Notebook を参照してください。

### 4-2. エンドポイントの削除

SageMaker 推論エンドポイントは起動したままだとコストがかかります。不要な場合は削除します。

In [12]:
xgb_predictor.delete_endpoint()

---
## 5. Notebook を移行する

社内のデータを扱う場合、 Studio Lab ではセキュリティが気になることがあるかもしれません。 Studio Lab から SageMaker Studio へ移行することで、データやネットワークのセキュリティを自社のセキュリティ基準に合わせて構築することができます。 SageMaker Studio へ移行することで、 Data Wrangler や SageMaker Pipeline を使用して Studio Lab では難しい規模のデータ処理ワークフローを構築することもできます。 SageMaker Studio は、 Studio Lab のエンタープライズ版とイメージ頂くとよいと思います。ほぼ同じインタフェースで、Studio Lab にはない機能の利用、高パフォーマンスのインスタンスでの Notebook の実行ができます。

![sagemaker_studio_001.png](images/sagemaker_studio_001.png)

Studio Lab から SageMaker Studio へ移行する方法は [Export Amazon SageMaker Studio Lab environment to Amazon SageMaker Studio](https://docs.aws.amazon.com/ja_jp/sagemaker/latest/dg/studio-lab-use-migrate.html) で紹介されていますが、端的には Git リポジトリを経由して簡単に移行することができます。本節ではその手順を解説します。

### SageMaker Studio Domain を作成する。

移行先の SageMaker Studio 環境を作成します。 SageMaker Studio の特徴は、1) JupyterLab のUIを提供するアプリケーションサーバー、2) ノートブックを実行するコンピューティングインスタンス、3) データを保管するボリュームが疎結合になっていることです。1, 2, 3をまとめたアーキテクチャを **SageMaker Studio Domain** と呼んでいます。

![sagemaker_studio_002](images/sagemaker_studio_002.png)

今回は SageMaker Studio Domain を作成済みとして、 SageMaker Studio を開いたところからスタートします。

### SageMaker StudioにNotebookを移行する

はじめに GitHub からリポジトリを clone します。 Studio Lab と同じように Git の拡張がインストールされているので拡張を使用し簡単に clone することができます。clone を実行するサーバーは JupyterLab のサーバー (JupyterServer App) になります。 clone が終了したら、 `notebooks/scenario_churn/customer_churn.ipynb` を開きます。Notebook を開くためのインスタンス (KernelGateway App) のイメージは Conda があらかじめインストールされている `Data Science` を選択します。 

![sagemaker_studio_003.png](images/sagemaker_studio_003-1.png)

![sagemaker_studio_003.png](images/sagemaker_studio_003-2.png)

JupyterServer App で clone を実行しましたが、 KernelGateway App でも clone したファイルを参照することができます。これは、 EFS でファイルを共有しているためです。

![sagemaker_studio_004.png](images/sagemaker_studio_004.png)

現状では環境が作成されていないので、イメージのターミナル (`Image Terminal`) を開いて環境を構築します。

![sagemaker_studio_005.png](images/sagemaker_studio_005.png)

`Image Terminal` で実行するコマンドは次の 3 つです。`notebooks/scenario_churn`のフォルダに移動して実行します。Studio Lab では右クリックだけで Conda の環境が作成できましたが、 Studio にはその機能がないため `Image Terminal` でコマンドを使い環境を作成します。

1. `conda env create -f environment.yml`
2. `conda activate ml-handson-churn`
3. `python -m ipykernel install`

1 番目のコマンドで環境を作成し、 2 番目のコマンドで作成した `ml-handson-churn` の環境を有効化し 3 番目のコマンドで Notebook から作成した環境が使えるようにしています。

![sagemaker_studio_006.png](images/sagemaker_studio_006.png)

全てのコマンドが実行した後、 Notebook を開きます。すると、 Notebook から作成した Kernel が選べるようになっています。

![sagemaker_studio_007.png](images/sagemaker_studio_007.png)

後の実行方法は Studio Lab と同じです。

### SageMaker Studio と Studio Lab の違い

SageMaker Studio で起動しているインスタンスは、一度シャットダウンすると再度環境構築が必要です。これは、次に立ち上がるインスタンスは新しいインスタンスであるためです。 Studio Lab では次に起動したとき環境が維持されていたので、手間がかかる点です。起動しているインスタンスは左側のメニューから確認できます。

![sagemaker_studio_009.png](images/sagemaker_studio_009.png)

起動に手間がかかりますが、必要に応じてインスタンスタイプを変えたり複数のインスタンスで Notebook を並行で実行するなど、クラウドならではのスケーラビリティを活かすことができます。

![sagemaker_studio_010.png](images/sagemaker_studio_010.png)

GPU インスタンス起動時にエラーが発生する場合、 GPU インスタンスが起動できる上限が 0 になっている可能性があります。 AWS マネジメントコンソールの [Service Quotas](https://console.aws.amazon.com/servicequotas/home/services/sagemaker/quotas) の画面から、制限を選択し必要な分を上限緩和申請してください。利用中のリージョンが選択されていることを確認の上、SageMaker Studio Notebook の GPU インスタンスであれば、例えば `Studio KernelGateway Apps running on ml.g4dn.xlarge instance` を選択してください。

![sagemaker_studio_011.png](images/sagemaker_studio_011.png)

Studio Lab での検証に課題があるとき、素早く AWS へ移行できることはプロジェクトの継続性の担保はもちろん、本番運用までのスピード向上にも効果的です。